## K-Means Station Clustering

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans

from deepar_model_utils import prep_station_data
from deepar_model_utils import get_station_data
from deepar_model_utils import get_cluster_data
from deepar_model_utils import deepar_station_data
from deepar_model_utils import write_dicts_to_file

%matplotlib inline

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


## Trip Start Station

In [2]:
start_file = "../model_trips_start_station_20208029_20220831.csv"

#s3_start_location = f"s3://{bucket}/{start_file}*"
#trips_start = pd.read_csv(s3_start_location, parse_dates = True)

trips_start = pd.read_csv(start_file, parse_dates = True)
trips_start.shape

(6059710, 2)

In [3]:
trips_start_all_group = prep_station_data(trips_start, "start station id", "starttime")
print(sum(trips_start_all_group["size"]))

6059710


In [4]:
freq = "15min" # group and sum trips by a set increment
min_date = "2020-08-29 00:00:00" # make sure all series end at the same time
max_date = "2022-08-31 23:45:00" # make sure all series end at the same time

In [5]:
start_list = []
for station in tqdm(trips_start_all_group["start station id"].unique()):
    start_list.append(get_cluster_data(trips_start_all_group, "start station id", "starttime", station, freq, min_date, max_date))

100%|██████████| 451/451 [00:38<00:00, 11.64it/s]


In [6]:
print(len(start_list))
print(sum([sum(i) for i in zip(*start_list)]))

451
6059710


In [7]:
formatted_start_list = to_time_series_dataset(start_list)
formatted_start_list.shape

(451, 70368, 1)

In [ ]:
%%time
start_km = TimeSeriesKMeans(n_clusters = 21, metric = "dtw", max_iter = 10)
start_km_labels = start_km.fit_predict(formatted_start_list)